In [1]:
import random
from collections import Counter
import stox
from ib_insync import *
from stox.imports import *
from stox.api import IB_Extractor, describe_contract, download_reports
from tqdm.notebook import tqdm
from omnibelt import tqdmd_notebook as tqdmd
from stox import misc, yahoo

In [2]:
symbol_table = stox.load_symbol_table()
len(symbol_table)

272

In [3]:
date = '230829'
# date = 'last'

In [4]:
yfroot = misc.yahoo_root()
ibroot = misc.ibkr_root()
# yfroot = yfroot.parent.parent / 'testdata' / 'yahoo_data'
# ibroot = ibroot.parent.parent / 'testdata' / 'ib_data'
yfroot.mkdir(exist_ok=True, parents=True)
ibroot.mkdir(exist_ok=True, parents=True)
print(yfroot)
print(ibroot)

C:\Users\anwan\OneDrive\Khan\projects\stox\stockdata\yahoo_data
C:\Users\anwan\OneDrive\Khan\projects\stox\stockdata\ibkr_data


In [5]:
rows = [(k, v) for k, v in symbol_table.items() if v['currency'] == 'EUR']
len(rows)

239

In [ ]:
# keys = dict(yahoo._default_datatypes)
# del keys['institutional_holders']
# del keys['major_holders']
# del keys['mutualfund_holders']
# idx = 0

In [142]:
yfsym, ibrow = random.choice(rows)

# manual = ['VIO']

# yfsym = bad_yf[idx]
# yfsym = bad_ib[idx]
# ibrow = symbol_table[yfsym]
# idx += 1

print(yfsym)#, ibrow

AD.AS


In [84]:
# tk = yahoo.download(yfsym, date=date, root=yfroot, pbar=tqdm, keys=keys)
tk = yahoo.download(yfsym, date=date, root=yfroot, pbar=tqdm)
tk

  0%|          | 0/11 [00:00<?, ?it/s]

<ZAL.DE>

In [9]:
ibe = IB_Extractor(True)
ib = ibe.ib

ERROR:ib_insync.wrapper: Error 321, reqId 2147483647: Error validating request.-'cG' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.wrapper: Error 321, reqId -1: Error validating request.-'ce' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.ib: open orders request timed out
ERROR:ib_insync.ib: completed orders request timed out


In [33]:
path = download_reports(ibe, ibrow, date=date, root=ibroot, pbar=tqdm)

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
# ct = Stock(**ibrow)
# print(' | '.join(map(str, [ct.symbol, ct.currency, ct.primaryExchange, ct.conId])))

AMS | EUR | BM | 74792133


In [137]:
bad_yf, bad_ib = [], []
for yfsym, ibrow in tqdmd(symbol_table.items(), total=len(symbol_table), key=lambda x: x[1]['symbol']):
	try:
		tk = yahoo.download(yfsym, date=date, root=yfroot, pbar=None)
	except KeyboardInterrupt:
		raise
	except:
		bad_yf.append(yfsym)
		print('Failed to download yahoo:', yfsym)
		continue
	if not ibe.ib.isConnected():
		ibe = IB_Extractor(True)
	try:
		path = download_reports(ibe, ibrow, date=date, root=ibroot, pbar=None)
	except KeyboardInterrupt:
		raise
	except:
		bad_ib.append(yfsym)
		print('Failed to download IBKR:', yfsym)
		continue

  0%|          | 0/272 [00:00<?, ?it/s]

ADYEN.AS institutional_holders None
Failed to download yahoo: ADYEN.AS
APAM.BR isin None
Failed to download yahoo: APAM.BR
ARGX.BR institutional_holders None
Failed to download yahoo: ARGX.BR
BELA.AT institutional_holders None
Failed to download yahoo: BELA.AT
CABK.MC institutional_holders None
Failed to download yahoo: CABK.MC
EBS.VI institutional_holders None
Failed to download yahoo: EBS.VI
HTO.AT institutional_holders None
Failed to download yahoo: HTO.AT
KER.PA institutional_holders None
Failed to download yahoo: KER.PA
LR.PA institutional_holders None
Failed to download yahoo: LR.PA
OR.PA institutional_holders None
Failed to download yahoo: OR.PA
PRY.MI institutional_holders None
Failed to download yahoo: PRY.MI
RACE.MI institutional_holders None
Failed to download yahoo: RACE.MI
SAR.AT institutional_holders None
Failed to download yahoo: SAR.AT
STR.VI institutional_holders None
Failed to download yahoo: STR.VI
UMI.BR institutional_holders None
Failed to download yahoo: UMI.BR
ZA

In [138]:
len(bad_yf), len(bad_ib)

(16, 0)

In [139]:
print(bad_ib) # ['ANE.MC', 'BPE.MI', 'CENER.AT', 'EVN.VI', 'FER.MC', 'HLN.L', 'LIN.DE', 'ML.PA', 'P911.DE', 'QUEST.AT', 'SDR.L', 'SHEL.L', 'SHELL.AS', 'SPM.MI', 'STERV.HE', 'UQA.VI', 'VIO.AT']

[]


In [140]:
print(bad_yf) # ['ADYEN.AS', 'APAM.BR', 'ARGX.BR', 'BELA.AT', 'CABK.MC', 'EBS.VI', 'HTO.AT', 'KER.PA', 'LR.PA', 'OR.PA', 'PRY.MI', 'RACE.MI', 'SAR.AT', 'STR.VI', 'UMI.BR', 'ZAL.DE']

['ADYEN.AS', 'APAM.BR', 'ARGX.BR', 'BELA.AT', 'CABK.MC', 'EBS.VI', 'HTO.AT', 'KER.PA', 'LR.PA', 'OR.PA', 'PRY.MI', 'RACE.MI', 'SAR.AT', 'STR.VI', 'UMI.BR', 'ZAL.DE']


In [ ]:
# unused

In [16]:
report = ibe.snapshot(ct)

In [ ]:
bysym = {v['symbol']: v for k, v in symbol_table.items()}
len(bysym)

In [16]:
row = next(iter(symbol_table.values()))
row = bysym['1COV']
row

{'conId': 208908717,
 'currency': 'EUR',
 'primaryExchange': 'IBIS',
 'symbol': '1COV'}

In [41]:
# fix = ib.reqContractDetails(Stock(**row))[0].contract
options = ib.reqContractDetails(Stock(
	# currency='EUR', 
	symbol='ZAL',
))

In [42]:
print(tabulate([(i, c.contract.symbol, c.contract.currency, c.contract.primaryExchange, c.contract.conId, c.contract.description) 
				for i, c in reversed(list(enumerate(options)))]))

--  ---  ---  ----  ---------
18  ZAL  CHF  EBS   172604484
17  ZAL  CHF  EBS   172604484
16  ZAL  NOK  OSE   158032384
15  ZAL  NOK  OSE   158032384
14  ZAL  NOK  OSE   158032384
13  ZAL  EUR  IBIS  168731675
12  ZAL  EUR  IBIS  168731675
11  ZAL  EUR  IBIS  168731675
10  ZAL  EUR  IBIS  168731675
 9  ZAL  EUR  IBIS  168731675
 8  ZAL  EUR  IBIS  168731675
 7  ZAL  EUR  IBIS  168731675
 6  ZAL  EUR  IBIS  168731675
 5  ZAL  EUR  IBIS  168731675
 4  ZAL  EUR  IBIS  168731675
 3  ZAL  EUR  IBIS  168731675
 2  ZAL  EUR  IBIS  168731675
 1  ZAL  EUR  IBIS  168731675
 0  ZAL  EUR  IBIS  168731675
--  ---  ---  ----  ---------


In [43]:
fix = options[0].contract
fix

Contract(secType='STK', conId=168731675, symbol='ZAL', exchange='SMART', primaryExchange='IBIS', currency='EUR', localSymbol='ZAL', tradingClass='XETRA')

In [44]:
fix.primaryExchange == row['primaryExchange'], fix.currency == row['currency']

(True, True)

In [45]:
row['exchange'] = fix.exchange

In [46]:
bad = []
for row in tqdmd(symbol_table.values(), key=lambda x: x['symbol']):
	if 'exchange' not in row:
		q = Stock(currency=row['currency'], 
				  # primaryExchange=row['primaryExchange'], 
				  symbol=row['symbol'])
		cts = [cd.contract for cd in ib.reqContractDetails(q) 
			   if cd.contract.conId == row['conId'] 
			   and cd.contract.primaryExchange == row['primaryExchange']]
		if len(cts):
			ct = cts[0]
			row['exchange'] = ct.exchange
		else:
			bad.append(row)

  0%|          | 0/272 [00:00<?, ?it/s]

In [31]:
len(bad), [b['symbol'] for b in bad]

(4, ['WIE', 'WKL', 'WLN', 'ZAL'])

In [47]:
# stox.save_symbol_table(symbol_table)

WindowsPath('C:/Users/anwan/OneDrive/Khan/projects/stox/assets/yahoo2ibkr.yml')